In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load your data
df_1 = pd.read_csv("jobs_in_data.csv")
# prefrom one hot encoding
df = pd.get_dummies(df_1)
df['salary_in_usd'] /= 1000
df['salary'] /= 1000


# Define features and target variable
# X_numerical = df.select_dtypes(include=np.number).drop(columns=['salary','salary_in_usd'])  # Select only numeric columns
X_numerical = df.drop(columns=['salary','salary_in_usd'])
y = df["salary_in_usd"]

# Scale numerical data
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)

# Split data into train and test sets
X_train_num, X_test_num, y_train, y_test = train_test_split(X_numerical_scaled, y, test_size=0.2, random_state=42)

# Reshape the input data to include timestep dimension
X_train_reshaped = X_train_num.reshape(X_train_num.shape[0], X_train_num.shape[1], 1)
X_test_reshaped = X_test_num.reshape(X_test_num.shape[0], X_test_num.shape[1], 1)

# Define LSTM model
input_layer = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
lstm_layer1 = LSTM(64, return_sequences=True)(input_layer)
dropout_layer1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = LSTM(64)(dropout_layer1)
dropout_layer2 = Dropout(0.2)(lstm_layer2)
output_layer = Dense(1, activation='linear')(dropout_layer2)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with a lower learning rate
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model with more epochs
model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", np.sqrt(mse))


Epoch 1/50
211/211 [==============================] - 68s 292ms/step - loss: 22103.6230 - val_loss: 20112.3770
Epoch 2/50
211/211 [==============================] - 75s 357ms/step - loss: 18476.0820 - val_loss: 17087.8145
Epoch 3/50
211/211 [==============================] - 69s 327ms/step - loss: 15712.1338 - val_loss: 14546.9629
Epoch 4/50
211/211 [==============================] - 70s 332ms/step - loss: 13358.9131 - val_loss: 12400.0576
Epoch 5/50
211/211 [==============================] - 65s 307ms/step - loss: 11410.1299 - val_loss: 10603.4629
Epoch 6/50
211/211 [==============================] - 70s 333ms/step - loss: 9758.1309 - val_loss: 9111.9453
Epoch 7/50
211/211 [==============================] - 71s 336ms/step - loss: 8407.3184 - val_loss: 7893.2778
Epoch 8/50
211/211 [==============================] - 62s 296ms/step - loss: 7328.7896 - val_loss: 6904.6313
Epoch 9/50
211/211 [==============================] - 67s 319ms/step - loss: 6444.9609 - val_loss: 6119.5073
Epoch 10/

164983.9573371127
39.736556370824324
11.114768167172112
63.93511358183519 -2000
63.934037317068835 -50
63.96655129502367 - one hot
63.91029864854856